In [ ]:
!pip install ultralytics

^C


     ------------------------------------- 887.4/887.4 kB 11.2 MB/s eta 0:00:00
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
     -------------------------------------- 162.0/162.0 kB 4.9 MB/s eta 0:00:00
     ------------------------------------- 203.1/203.1 MB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 33.0 MB/s eta 0:00:00
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
     ------------------------------------- 179.6/179.6 kB 11.3 MB/s eta 0:00:00
  Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [ ]:
import torch
print(torch.cuda.is_available())
print (torch.version.cuda)

In [14]:
#This segment converts polygon to bounding boxes as yolov8 only allows for bounding boxes for object detection. 
#How to solve this?

#2 ways 
#Either I convert all of bounding boxes to segmentation 
#Convert all segmentation to bounding boxes. 
import os
def bounding_box_to_polygon(labels_folder):
    for filename in os.listdir(labels_folder):
        if filename.endswith('.txt'):
            filepath = os.path.join(labels_folder, filename)

            with open(filepath, 'r') as file:
                lines = file.readlines()

            updated_lines = []
            for line in lines:
                elements = line.strip().split()
                if len(elements) == 5:
                    _, x, y, width, height = map(float, elements)
                    x_min = x - width/2
                    x_max = x + width/2
                    y_min = y - height/2
                    y_max = y + height/2
                    updated_elements = [elements[0], str(x_min), str(y_min), str(x_max), str(y_min), str(x_max), str(y_max), str(x_min), str(y_max)]
                
                    updated_lines.append(' '.join(updated_elements))

            with open(filepath, 'w') as file:
                file.write('\n'.join(updated_lines))

file_List = ["test","valid", "train"]

for i in file_List:
    path = r"car-models-on-expressways-12/{}/labels".format(i)

# file_list = ["train"]
# path = r"car-models-on-expressways-10/train/labels"
print(path)
bounding_box_to_polygon(path)

car-models-on-expressways-12/train/labels


In [1]:
#Add K fold Cross Validation
#This attempts tries to work K fold only the training set. 
from pathlib import Path
from ultralytics import YOLO
import yaml
import pandas as pd
from collections import Counter
import datetime 
dataset_path = Path("./car-models-on-expressways-10/train")
labels = sorted(dataset_path.glob("labels/*.txt"))

yaml_file = "car-models-on-expressways-10/data.yaml"  # your data YAML with data directories and names dictionary
with open(yaml_file, "r", encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]

cls_idx=[]
for i in range(len(classes)):
    cls_idx.append(i)
print(cls_idx)
indx = [l.stem for l in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)
print(labels_df)
for label in labels:
    lbl_counter = Counter()
    with open(label, "r") as lf:
        lines = lf.readlines()
    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(" ")[0])] += 1
    labels_df.loc[label.stem] = lbl_counter

#     for key, value in lbl_counter.items():
#         labels_df.loc[label.stem, key] = value

labels_df = labels_df.fillna(0.0)
print(labels_df)
        
    
#     row_data=dict(lbl_counter)
#     index_to_column = {0: 'bike', 1: 'bus', 2: 'car', 3: 'lorry'}
#     for key, value in row_data.items():
#         if key in index_to_column:
#             column = index_to_column[key]
#             labels_df.loc[label.stem, column] = value



# labels_df = labels_df.fillna(0.0)  
# print(labels_df)


[0, 1, 2, 3, 4]
                                                      0    1    2    3    4
1001-East-Coast-Parkway-1-29531332-103-871146-2...  NaN  NaN  NaN  NaN  NaN
1001-East-Coast-Parkway-1-29531332-103-871146-2...  NaN  NaN  NaN  NaN  NaN
1001-East-Coast-Parkway-1-29531332-103-871146-2...  NaN  NaN  NaN  NaN  NaN
1001-East-Coast-Parkway-1-29531332-103-871146-2...  NaN  NaN  NaN  NaN  NaN
1001-East-Coast-Parkway-1-29531332-103-871146-2...  NaN  NaN  NaN  NaN  NaN
...                                                 ...  ...  ...  ...  ...
9706-Seletar-Expressway-1-41270056-103-80642712...  NaN  NaN  NaN  NaN  NaN
9706-Seletar-Expressway-1-41270056-103-80642712...  NaN  NaN  NaN  NaN  NaN
9706-Seletar-Expressway-1-41270056-103-80642712...  NaN  NaN  NaN  NaN  NaN
9706-Seletar-Expressway-1-41270056-103-80642712...  NaN  NaN  NaN  NaN  NaN
9706-Seletar-Expressway-1-41270056-103-80642712...  NaN  NaN  NaN  NaN  NaN

[3549 rows x 5 columns]
                                               

In [3]:
from sklearn.model_selection import KFold,  StratifiedKFold
ksplit = 5

value  = "sk"
if value == "k":
    kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results
    kfolds = list(kf.split(labels_df))
elif value=="sk":
    #TESTING OUT STRATIFIEDKFOLD TO ENSURE A BALANCED DATASET
    skf = StratifiedKFold(n_splits=ksplit, shuffle=True, random_state=42)
    kfolds = list(skf.split(labels_df, labels_df.idxmax(axis=1)))
    

folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)



for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f"split_{idx}"].loc[labels_df.iloc[train].index] = "train"
    folds_df[f"split_{idx}"].loc[labels_df.iloc[val].index] = "val"
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)
for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()
    
    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

print(fold_lbl_distrb)
    

                0         1         2         3         4
split_1  0.242797  0.239901  0.232456  0.241931  0.285054
split_2  0.231601  0.249132  0.249313  0.246303  0.219298
split_3  0.263832  0.278314  0.254718  0.248061  0.239227
split_4  0.269791  0.229004  0.259703  0.270457  0.250375
split_5  0.242797  0.254736  0.254167  0.243676  0.257919


In [6]:
import datetime
supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []
img_dataset_path = Path("car-models-on-expressways-10/train/")
# Loop through supported extensions and gather image files
# for ext in supported_extensions:
images.extend(sorted((img_dataset_path).glob("*images/*.jpg")))

    
# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path/ f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)
    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )

In [7]:
import shutil
from pathlib import Path


for image, label in zip(images, labels):

    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory

        img_to_path = save_path / split / k_split / "images"
        lbl_to_path = save_path / split / k_split / "labels"
        
        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

In [ ]:
#Code to start 5k fold cross validation with a smaller model of yolov8-L instead of yolov8-x and probably increase epoch size. 
from ultralytics import YOLO
temp_yamls = Path("car-models-on-expressways-10/train/2024-06-30_5-Fold_Cross-val")
test_yamls=[]
test_yamls.extend(sorted((temp_yamls).rglob("*.yaml")))
# print(test_yamls)
# print("break")
# print(ds_yamls)
results = {}
project = "5kfold_demo"
epochs = 250
for k in range(ksplit):
    model = YOLO("yolov8x-seg.pt")
    dataset_yaml = test_yamls[k]
    model.train(data=dataset_yaml, epochs=epochs, project=project)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis
    
print(results)




New https://pypi.org/project/ultralytics/8.2.48 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.134 🚀 Python-3.7.6 torch-1.13.1+cu117 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
yolo/engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=car-models-on-expressways-10/train/2024-06-30_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=250, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=5kfold_demo, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=F

In [ ]:
#Code to resume training for 5k fold validation
from ultralytics import YOLO
from pathlib import Path
import yaml
import pandas as pd
from collections import Counter
import datetime 
path= "5kfold_demo"
import os


def file_exists(filename, directory):
    return os.path.isfile(os.path.join(directory, filename))
# Get the current working directory
current_dir = path

ksplit=5
# List all subdirectories in the current directory
subdirectories = [d for d in os.listdir(current_dir) if os.path.isdir(os.path.join(current_dir, d))]
if(".ipynb_checkpoints" in subdirectories):
    subdirectories.remove(".ipynb_checkpoints")
lasttrain = subdirectories[-1]
print(subdirectories, lasttrain)
directory = path+"/"+lasttrain

if(file_exists("results.png",directory) == False):
    print("pass")
    path= "5kfold_demo/{}/weights/best.pt".format(lasttrain)
    model =YOLO(path)
    results = model.train(resume=True)
    


temp_yamls = Path("car-models-on-expressways-10/train/2024-06-30_5-Fold_Cross-val")
test_yamls=[]
test_yamls.extend(sorted((temp_yamls).rglob("*.yaml")))
print(test_yamls)
results = {}
project = "5kfold_demo"
epochs = 250
for k in range(len(subdirectories),ksplit):
    print(test_yamls[k])
    model = YOLO("yolov8x-seg.pt")
    dataset_yaml = test_yamls[k]
    model.train(data=dataset_yaml, epochs=epochs, project=project)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis



# print(results)





['train'] train
pass


New https://pypi.org/project/ultralytics/8.2.48 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.134 🚀 Python-3.7.6 torch-1.13.1+cu117 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
yolo/engine/trainer: task=segment, mode=train, model=5kfold_demo/train/weights/best.pt, data=car-models-on-expressways-10/train/2024-06-30_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=250, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=5kfold_demo, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt

In [ ]:
#Code to resume training for 9k fold validation
from ultralytics import YOLO
from pathlib import Path
import yaml
import pandas as pd
from collections import Counter
import datetime 
path= "10kfold_demo"
import os


def file_exists(filename, directory):
    return os.path.isfile(os.path.join(directory, filename))
# Get the current working directory
current_dir = path

ksplit=9
# List all subdirectories in the current directory
subdirectories = [d for d in os.listdir(current_dir) if os.path.isdir(os.path.join(current_dir, d))]
if(".ipynb_checkpoints" in subdirectories):
    subdirectories.remove(".ipynb_checkpoints")
lasttrain = subdirectories[-1]
print(subdirectories, lasttrain)
directory = path+"/"+lasttrain

if(file_exists("results.png",directory) == False):
    print("pass")
    path= "10kfold_demo/{}/weights/best.pt".format(lasttrain)
    model =YOLO(path)
    results = model.train(resume=True)
    


temp_yamls = Path("car-models-on-expressways-10/train/2024-06-26_9-Fold_Cross-val")
test_yamls=[]
test_yamls.extend(sorted((temp_yamls).rglob("*.yaml")))
print(test_yamls)
results = {}
project = "10kfold_demo"
epochs = 150
for k in range(len(subdirectories),ksplit):
    print(test_yamls[k])
    model = YOLO("yolov8x-seg.pt")
    dataset_yaml = test_yamls[k]
    model.train(data=dataset_yaml, epochs=epochs, project=project)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis

    

In [ ]:
#Code to initiate 9k with yolov8-N
from ultralytics import YOLO
temp_yamls = Path("car-models-on-expressways-10/train/2024-06-26_9-Fold_Cross-val")
test_yamls=[]
test_yamls.extend(sorted((temp_yamls).rglob("*.yaml")))
# print(test_yamls)
# print("break")
# print(ds_yamls)
results = {}
project = "10kfold_demo"
epochs = 150
for k in range(ksplit):
    model = YOLO("yolov8x-seg.pt")
    dataset_yaml = test_yamls[k]
    model.train(data=dataset_yaml, epochs=epochs, project=project)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis
    
print(results)

New https://pypi.org/project/ultralytics/8.2.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.134 🚀 Python-3.7.6 torch-1.13.1+cu117 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
yolo/engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=car-models-on-expressways-10/train/2024-06-26_9-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=150, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=10kfold_demo, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=

In [ ]:
#Code to resume training for 9k fold validation
from ultralytics import YOLO
from pathlib import Path
import yaml
import pandas as pd
from collections import Counter
import datetime 
path= "10kfold_demo"
import os


def file_exists(filename, directory):
    return os.path.isfile(os.path.join(directory, filename))
# Get the current working directory
current_dir = path

ksplit=9
# List all subdirectories in the current directory
subdirectories = [d for d in os.listdir(current_dir) if os.path.isdir(os.path.join(current_dir, d))]
if(".ipynb_checkpoints" in subdirectories):
    subdirectories.remove(".ipynb_checkpoints")
lasttrain = subdirectories[-1]
print(subdirectories, lasttrain)
directory = path+"/"+lasttrain

if(file_exists("results.png",directory) == False):
    print("pass")
    path= "10kfold_demo/{}/weights/best.pt".format(lasttrain)
    model =YOLO(path)
    results = model.train(resume=True)
    


temp_yamls = Path("car-models-on-expressways-10/train/2024-06-26_9-Fold_Cross-val")
test_yamls=[]
test_yamls.extend(sorted((temp_yamls).rglob("*.yaml")))
print(test_yamls)
results = {}
project = "10kfold_demo"
epochs = 150
for k in range(len(subdirectories),ksplit):
    print(test_yamls[k])
    model = YOLO("yolov8x-seg.pt")
    dataset_yaml = test_yamls[k]
    model.train(data=dataset_yaml, epochs=epochs, project=project)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis

    









# print(results)





['train', 'train1', 'train2', 'train3', 'train4', 'train5'] train5
pass


New https://pypi.org/project/ultralytics/8.2.46 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.134 🚀 Python-3.7.6 torch-1.13.1+cu117 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
yolo/engine/trainer: task=segment, mode=train, model=10kfold_demo/train5/weights/best.pt, data=car-models-on-expressways-10/train/2024-06-26_9-Fold_Cross-val/split_6/split_6_dataset.yaml, epochs=150, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=10kfold_demo, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_